In [12]:
import folium #to create the map | pip install folium
from quakefeeds import QuakeFeed #to get information about US GOV earthquakes (it is storaged for 1 day, 7 days or 1 month) | pip install quakefeeds
from folium.plugins import HeatMap, MousePosition, MiniMap, MeasureControl, Fullscreen, TagFilterButton #get some map interactions
from datetime import datetime #to get some datitime formats | pip install datetime
import pandas as pd #to manipulate data as dataframe | pip install pandas
import numpy as np #to manipulate data and do some calculations | pip install numpy

In [13]:
feed = QuakeFeed("4.5", "month") #load data of earthquakes with magnitude grater than 4.5 (it is the minimum magnitude)
#and events that were captured last month (maximum time interval)

In [14]:
places = [x["properties"]['place'] for x in feed] #extract place of every event
magnitudes = [x["properties"]['mag'] for x in feed] #extract the magnitude of every event
years = [datetime.utcfromtimestamp(round(x["properties"]['time']/1000)).year for x in feed] #extract year of event
months = [datetime.utcfromtimestamp(round(x["properties"]['time']/1000)).strftime("%B") for x in feed] #extract month of event
days = [datetime.utcfromtimestamp(round(x["properties"]['time']/1000)).day for x in feed] #extract day of event
locations = [list(reversed(x["geometry"]["coordinates"][0:2])) for x in feed] #extract the coordinates of the event (reversed because they come as lon, lat)

In [15]:
df = pd.DataFrame({"Place": places, #create a dataframe with the list defined before
                   "Magnitude": magnitudes,
                   "Year": years,
                   "Month": months,
                   "Day": days,
                   "Location": locations})

In [16]:
heat_data = [[row['Location'][0], row["Location"][1], row['Magnitude']] for index, row in df.iterrows()] #this is to work heatmap because data 
#should be in format [lat, lon, value] to properly use HeatMap function from folium plugins

In [17]:
df["color"] = np.where(df["Magnitude"] <= 5, "green", #just add categorical colors to data depending on magnitude
              np.where(df["Magnitude"] <= 6, "orange", "red"))

In [19]:
map = folium.Map(tiles = 'OpenStreetMap', zoom_start = 19, max_bounds=True, max_bounds_viscosity=1.0) #create folium map (format using OpenStreetMap)


red_markers = folium.FeatureGroup(name='High Magnitude') #create a layer for 'red' events (using color variable)
orange_markers = folium.FeatureGroup(name='Mid Magnitude') #create a layer for 'orange' events (using color variable, yellow was not available)
green_markers = folium.FeatureGroup(name='Low Magnitude') #create a layer for 'green' events (using color variable)


categories = list(df["Year"].unique().astype(str)) #create categories to make tags on datapoints (use to create filters, Year, , is important to cast as string)
categories2 = list(df["Month"].unique().astype(str)) #create categories to make tags on datapoints (use to create filters, Month, is important to cast as string)
categories3 = list(np.sort(df["Day"].unique()).astype(str)) #create categories to make tags on datapoints (use to create filters, day, sorted, is important to cast as string)

In [20]:
for i in range(len(df)):#iterate over every df rows
    if df["color"][i] == "red":  #if color is red we add it to red layer
        folium.Marker(location = df["Location"][i], #coordinates
              popup = folium.Popup('''
            <h5 style="text-align: center;"><b>{name}</b></h5>
            <h6 style="text-align: center;"><b> Magnitude: {mag}</b></h6>
                <ul>
                <b><li> Year: {year} </li> </b>
                <b><li> Month: {month} </li> </b>
                <b><li> Day: {day} </li> </b>
                </ul>
                                '''.format(name = df["Place"][i], year = df["Year"][i], month = df["Month"][i], day = df["Day"][i], mag = df["Magnitude"][i]),    max_width=300), #this used to show some information of the event using HTML formatin and programming
            icon=folium.Icon(color=df["color"][i]), #make the icon of the point match layer color (icon can be change using 'fa-icons')
            tags = [str(df["Year"][i]), df["Month"][i], str(df["Day"][i])] #add the tags
            ).add_to(red_markers) #finally, ad to desire layer
        
    elif df["color"][i] == "orange": #if color is orange we add it to orange layer
        folium.Marker(location = df["Location"][i], #coordinates
              popup = folium.Popup('''
            <h5 style="text-align: center;"><b>{name}</b></h5>
            <h6 style="text-align: center;"><b> Magnitude: {mag}</b></h6>
                <ul>
                <b><li> Year: {year} </li> </b>
                <b><li> Month: {month} </li> </b>
                <b><li> Day: {day} </li> </b>
                </ul>
                                '''.format(name = df["Place"][i], year = df["Year"][i], month = df["Month"][i], day = df["Day"][i],  mag = df["Magnitude"][i]), max_width=300), #this used to show some information of the event using HTML formatin and programming
            icon=folium.Icon(color=df["color"][i]), #make the icon of the point match layer color (icon can be change using 'fa-icons')
            tags = [str(df["Year"][i]), df["Month"][i], str(df["Day"][i])] #add the tags
            ).add_to(orange_markers) #finally, ad to desire layer
        
    else: #else it belongs to green layer
        folium.Marker(location = df["Location"][i], #coordinates
              popup = folium.Popup('''
            <h5 style="text-align: center;"><b>{name}</b></h5>
            <h6 style="text-align: center;"><b> Magnitude: {mag}</b></h6>
                <ul>
                <b><li> Year: {year} </li> </b>
                <b><li> Month: {month} </li> </b>
                <b><li> Day: {day} </li> </b>
                </ul>
                                '''.format(name = df["Place"][i], year = df["Year"][i], month = df["Month"][i], day = df["Day"][i],  mag = df["Magnitude"][i]), max_width=300), #this used to show some information of the event using HTML formatin and programming
            icon=folium.Icon(color=df["color"][i]), #make the icon of the point match layer color (icon can be change using 'fa-icons')
            tags = [str(df["Year"][i]), df["Month"][i], str(df["Day"][i])] #add the tags
            ).add_to(green_markers) #finally, ad to desire layer
        
    
map.add_child(red_markers) #add red layer to the map
map.add_child(orange_markers) #add orange layer to the map
map.add_child(green_markers) #add green layer to the map

HeatMap(heat_data, name='Heatmap').add_to(map) #add heatmaplayer to the map with name 'Heatmap'

Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(map) #add plugin to add 'Fullscreen' feature to html map


folium.LayerControl(collapsed=True).add_to(map) #add layer control (so you can filter between layers)


map.add_child(MeasureControl()) #add measure plugin (you can measure distance if desired)

MousePosition().add_to(map) #this is to control the map with the mouse (hold click and move)
MiniMap().add_to(map) #this is to add 'Minimap' feature to the map

TagFilterButton(categories, icon= '<i class="fa fa-clock"></i>').add_to(map) #add filter for tags (Year)
TagFilterButton(categories2, icon= '<i class="fa fa-calendar"></i>').add_to(map) #add filters for tags (Month)
TagFilterButton(categories3, icon = '<i class="fa-solid fa-calendar-days"></i>').add_to(map) #add filter for tags (day)

In [21]:
map #show the map

In [10]:
map.save("C:/Otros Proyectos/earthquakes_map_visualization.html") #save map